## Data Loading
Loads in the big data matrix and the stfp matrix created by matlab

In [1]:
from scipy.io import loadmat

def load_in_data():
    file = scipy.io.loadmat("data/simulationData.mat")
    data = file["data"]
    return data

def load_stfp():
    file = scipy.io.loadmat("data/stfp.mat")
    stfp = file["stfp"]
    return stfp


## Data processing
Processes the data just as in the matlab version

In [2]:
# Set the notebook directory as the working directory
%pwd
%cd C:\Users\Rafael\Documents\GitHub\BeyondHulten

[Errno 2] No such file or directory: 'C:UsersRafaelDocumentsGitHubBeyondHulten'
/home/franzs/Schreibtisch/BeyondHulten


In [3]:
import numpy as np
import pandas as pd
import scipy

#data is the IO matrix for 88 sectors over 46 years (1960-2005)
data2 = load_in_data()
ind = pd.MultiIndex.from_arrays(data2[:, (0,1)].T, names=('year','sec'))
data = pd.DataFrame(data2[:, 2:93], index=ind)
del data2, ind

In [4]:
#Column 0 of datavector contains gross output for each sector
print(data.loc[(1960,2),0]*2) #first column times two
print(data.loc[(1960,2)].sum()) #summed across all columns (including the sum in column 0 already)
#Column 1 and 2 contain labour and capital
print(data.loc[(2005,2),0:2])
#Columns 3 to 90 contain the 88 sectors
#Colum 91 contains noncompetitive imports


3434.026
3433.7749999999996
0    88366.023
1    25623.402
2    31151.107
Name: (2005.0, 2.0), dtype: float64


In [5]:
#delete 10 sectors
removable_sectors = [59] + list(range(79, 88))
data2 = data.drop(index=removable_sectors, level=1)

#key aggregates
grossy = data2.loc[:,0].unstack().T
capital = data2.loc[:,1].unstack().T
labor = data2.loc[:,2].unstack().T
vadd = labor + capital

#remaining sector list
sec = list(range(1,89))
for i in removable_sectors:
    sec.remove(i)
len(sec)

#delete sectors which don't produce anything
grossy.loc[(grossy==0).all(axis=1)] #sectors 8 and 62
zeroprod = list(grossy.loc[(grossy==0).all(axis=1)].index.values)

grossy = grossy.drop(index=zeroprod)
capital = capital.drop(index=zeroprod)
labor = labor.drop(index=zeroprod)
vadd = vadd.drop(index=zeroprod)

stfp = load_stfp()
Σ = np.cov(stfp, rowvar=True)
μ = np.mean(stfp, axis=1)

## Creating usable variables
Function that creates the metrics for a given year

In [12]:
def get_variables(year):
    IO = data.loc[year,:]
    
    sectors_to_remove = list(map(int, grossy[year].index))
    temp = list(map(lambda x: x - 1 ,sectors_to_remove))
    IO = IO.loc[sectors_to_remove,temp]

    Ω = IO.div(IO.sum(axis = 1),axis = 0).to_numpy()
    
    α = vadd[year]/grossy[year]
    β = (np.eye(len(grossy)) - np.diag(1 - α) @ Ω).T @ grossy[year]
    β[β < 0] = 0
    β = β / np.sum(β)
    λ = np.linalg.inv( np.eye(len(grossy)) - np.diag(1 - α) @ Ω).T @ β
    L = λ * α

    return α, β, Ω, L, λ

## Elasticities

In [13]:
ε = .5;
θ = 0.001;
σ = .9;

# Construction Site Below HERE

## Objective Function

Here the objective function in `SimulationDerivs.m` is formulated, also the Jacobian is given, to helpt the solver.

In [42]:
def problem(X, A, β, Ω, α, ε, θ, σ, L):
    N = len(α)
    p = X[:N]
    y = X[N:]

    Out = np.zeros(2 * N, dtype=X.dtype)

    q = (Ω @ (p ** (1 - θ))) ** (1 / (1 - θ))
    w = p * (A ** ((ε - 1) / ε)) * (α ** (1 / ε)) * (y ** (1 / ε)) * L ** (-1 / ε)
    C = np.dot(w, L)

    Out[:N] = p - (A ** (ε - 1) * (α * w ** (1 - ε) + (1 - α) * q ** (1 - ε))) ** (1 / (1 - ε))
    Out[N:] = y - (y * p ** ε * A ** (ε - 1) * q ** (θ - ε) * (1 - α)) @ Ω @ (p ** - θ) - C * np.dot(β,p**-θ)  
    return Out


## Generating Random Shocks

In [43]:
from scipy.stats import multivariate_normal

def generate_random_shock(Σ, α, Ω, λ):
    n = len(Σ)
    cov_matrix = np.diag(np.diag(Σ))
    A = np.exp(multivariate_normal.rvs(mean=-0.5 * np.diag(Σ), cov=cov_matrix))
    
    inv_term = np.linalg.inv(np.eye(n) - np.diag(1 - α) @ Ω)
    log_A = np.log(A)
    init = np.concatenate((np.exp(-inv_term @ log_A), λ / np.exp(-inv_term @ log_A)))
    
    return A, init


## Solving for shocks

In [55]:
trials = 100
GDP = np.zeros(trials)
λ_sim = np.zeros((76, trials))
α, β, Ω, L, λ = get_variables(1973)

A = np.ones(76, dtype=np.complex128)
p = np.ones(76,dtype=np.complex128)
y = λ
init = np.concatenate((p,y))

sol = root(lambda x0 : problem(x0, A, β, Ω, α, ε, θ, σ, L), init, method='hybr')


TypeError: Cannot cast array data from dtype('complex128') to dtype('float64') according to the rule 'safe'

In [46]:
#This is from ChatGPT

import multiprocessing as mp
from scipy.optimize import root

# Parallel worker function
def parallel_worker(k):
    A, init = generate_random_shock(Σ, α, Ω, λ)
    p = [A, β, Ω, α, ε, θ, σ, L]

    sol = root(lambda x0 : problem(x0, A, β, Ω, α, ε, θ, σ, L), init, method='hybr')

    x = np.real(sol.x)
    p = x[:76]
    q = x[76:]
    gdp = np.dot(p * (A ** ((ε - 1) / ε)) * (α ** (1 / ε)) * (q ** (1 / ε)) * (L ** (-1 / ε)), L)
    λ_sim[:, k] = (p * q) / gdp
    return gdp

# Parameters
trials = 100
GDP = np.zeros(trials)
λ_sim = np.zeros((76, trials))

factor_share, consumption_share, io_matrix, labor_share, domar_weights = get_variables(1973)
# Parallel execution
num_processes = mp.cpu_count()
with mp.Pool(processes=num_processes) as pool:
    results = pool.map(parallel_worker, range(trials))
    GDP = np.array(results)

# Now you have both GDP and λ_sim populated after the parallel execution

/tmp/ipykernel_25048/3870089151.py:8: RuntimeWarning: invalid value encountered in power
  q = (Ω @ (p ** (1 - θ))) ** (1 / (1 - θ))
/tmp/ipykernel_25048/3870089151.py:8: RuntimeWarning: invalid value encountered in power
  q = (Ω @ (p ** (1 - θ))) ** (1 / (1 - θ))
/tmp/ipykernel_25048/3870089151.py:8: RuntimeWarning: invalid value encountered in power
  q = (Ω @ (p ** (1 - θ))) ** (1 / (1 - θ))
/tmp/ipykernel_25048/3870089151.py:13: RuntimeWarning: invalid value encountered in sqrt
  Out[N:] = y - (y * p ** ε * A ** (ε - 1) * q ** (θ - ε) * (1 - α)) @ Ω @ (p ** - θ) - C * np.dot(β,p**-θ)
/tmp/ipykernel_25048/3870089151.py:8: RuntimeWarning: invalid value encountered in power
  q = (Ω @ (p ** (1 - θ))) ** (1 / (1 - θ))
/tmp/ipykernel_25048/3870089151.py:8: RuntimeWarning: invalid value encountered in power
  q = (Ω @ (p ** (1 - θ))) ** (1 / (1 - θ))
/tmp/ipykernel_25048/3870089151.py:13: RuntimeWarning: invalid value encountered in sqrt
  Out[N:] = y - (y * p ** ε * A ** (ε - 1) * q **